In [ ]:
import numpy as np
import pandas as pd
from math import sqrt

In [ ]:

vali=pd.read_csv('./m5-forecasting-accuracy/sales_train_validation.csv')
c=pd.read_csv('./m5-forecasting-accuracy/calendar.csv')
eval=pd.read_csv('./m5-forecasting-accuracy/sales_train_evaluation.csv')
price_data=pd.read_csv('./m5-forecasting-accuracy/sell_prices.csv')
vali = vali.loc[vali['store_id'] == 'CA_1']#只选取一家店



In [ ]:
#取成为整数周的列，整合成
week_len=len(vali.columns[6:])//7*7
vali=vali.iloc[:,:6+week_len]
# 首先筛选所有日期列（d_1到d_n）
date_cols = [col for col in vali.columns if col.startswith('d_')]

# 按周聚合函数
def aggregate_weekly(df, date_cols):
    # #计算出年平均销量，不包含最后28天
    # annual_cols=date_cols[:-28]
    # df['annual_sales'] = df[annual_cols].sum(axis=1) * (365 / len(annual_cols))


    # 1. 计算总周数（向上取整）
    num_weeks = int(np.ceil(len(date_cols) / 7))
    
    # 2. 创建周销量列名
    weekly_cols = [f'weekly_{i+1}' for i in range(num_weeks)]
    
    # 3. 初始化周销量DataFrame
    weekly_df = pd.DataFrame(index=df.index)
    
    # 4. 按周聚合（每7天求和）
    for week_idx in range(num_weeks):
        start = week_idx * 7
        end = start + 7
        current_week_cols = date_cols[start:end]
        
        # 处理最后不足7天的情况
        if not current_week_cols:
            break
            
        weekly_df[weekly_cols[week_idx]] = df[current_week_cols].sum(axis=1)
    
    # 5. 合并非日期列
    non_date_cols = [col for col in df.columns if not col.startswith('d_')]
    result = pd.concat([df[non_date_cols], weekly_df], axis=1)
    
    return result

# 执行聚合
weekly_sales_df = aggregate_weekly(vali, date_cols)

# 验证结果
print("原始数据形状:", vali.shape)
print("周聚合后形状:", weekly_sales_df.shape)
print("\n前5行示例:")
print(weekly_sales_df.head())  

In [ ]:
#改成用week、id唯一确定一行的形式
# 假设 weekly_sales_df 是当前的数据框
# 步骤1：筛选出所有周销量列（weekly_*）
weekly_cols = [col for col in weekly_sales_df.columns if col.startswith('weekly_')]

# 步骤2：使用melt函数转换数据格式
long_format_df = pd.melt(
    weekly_sales_df,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],  # 需要保留的列
    value_vars=weekly_cols,
    var_name='week',
    value_name='weekly_sales'
)

# 步骤3：清理week列（提取数字部分）
long_format_df['week'] = long_format_df['week'].str.extract('(\d+)').astype(int)

# 步骤4：按id和week排序
long_format_df = long_format_df.sort_values(['id', 'week']).reset_index(drop=True)

# 结果预览
print(long_format_df.head(10))

In [ ]:
# 合并price中有用的数据
# 1. 预处理price数据（从变量c中提取）
price_data['week'] = price_data['wm_yr_wk'] - 11100  # 转换为与long_format_df一致的周编号

# 2. 选择需要的列（避免重复列）
price_data = price_data[['store_id', 'item_id', 'week', 'sell_price']]

# 3. 合并到long_format_df（注意多条件匹配）
merged_df = pd.merge(
    long_format_df,
    price_data,
    how='left',  # 保留所有销售记录，没有价格的行填充NaN
    on=['store_id', 'item_id', 'week']  # 三个匹配条件
)

# 4. 检查合并结果
print(f"合并前记录数: {len(long_format_df)}")
print(f"合并后记录数: {len(merged_df)}")
print("价格缺失记录数:", merged_df['sell_price'].isna().sum())
print("\n合并后示例:")
print(merged_df.head(10))

# 按ID分组统计非空售价数量
id_price_stats = merged_df.groupby('id')['sell_price'].agg(
    has_price= lambda x: x.notnull().any(),  # 是否存在至少一个非空值
    null_count= lambda x: x.isnull().sum()   # 该ID的空值数量
)

# 提取完全没有售价的ID列表
no_price_ids = id_price_stats[~id_price_stats['has_price']].index.tolist()
print(f"共有 {len(no_price_ids)} 个ID完全没有售价")
print("示例ID:", no_price_ids[:5])

# 步骤1：识别至少有一条sell_price非空的ID
valid_ids = merged_df[merged_df['sell_price'].notnull()]['id'].unique()

# 步骤2：仅保留这些ID的所有记录（包括其空值记录）
merged_df = merged_df[merged_df['id'].isin(valid_ids)]

# 验证结果
print(f"删除后剩余记录数: {len(merged_df)}")
print(f"剩余唯一ID数: {merged_df['id'].nunique()}")
print("sell_price空值数:", merged_df['sell_price'].isnull().sum())  # 此时应>0

In [ ]:

# 对calendar进行处理，保留的特征有活动数，cultural活动数、national、religous、sporting活动数、CA福利发放日数、TX福利发放日数、WI福利发放日数、周数为40-48周内

# 1. 处理wm_yr_wk列
c['wm_yr_wk'] = c['wm_yr_wk'] - 11100

# 2. 创建周特征数据框
# 首先按周分组
weekly_data = c.groupby('wm_yr_wk')

# 初始化week_char数据框
week_char = pd.DataFrame()

# 计算各周特征
week_char['event_count'] = weekly_data.apply(
    lambda x: (~x['event_name_1'].isna()).sum() + (~x['event_name_2'].isna()).sum()
)

# 计算各类活动数量
event_types = ['Cultural', 'Religious', 'National', 'Sporting']
for event_type in event_types:
    week_char[f'{event_type.lower()}_count'] = weekly_data.apply(
        lambda x: ((x['event_type_1'] == event_type) | (x['event_type_2'] == event_type)).sum()
    )

# 计算福利发放日总数
states = ['CA', 'TX', 'WI']
for state in states:
    week_char[f'snap_{state}_total'] = weekly_data[f'snap_{state}'].sum()

# 判断周数是否为11-12月
week_char['is_nov_dec'] = weekly_data['month'].apply(
    lambda x: x.isin([11, 12]).any()
).astype(int)

# 重置索引使wm_yr_wk成为列
week_char = week_char.reset_index()

# 显示结果
print(week_char.head())


week_char = week_char.rename(columns={'wm_yr_wk': 'week'})

price_date_data = pd.merge(merged_df, week_char, on='week', how='left')#合并了周的特征、价格的数据
print(price_date_data[['id','week']].head(275))


# 按商品分组的周销售统计
item_stats = price_date_data.groupby('id')['weekly_sales'].describe(percentiles=[.25, .5, .75])
print(item_stats)


null_counts = price_date_data.isnull().sum()
print("各列空值统计:")
print(null_counts[null_counts > 0])  # 只显示有空值的列
print(price_date_data.shape)

In [ ]:

unique_states = price_date_data['state_id'].unique()
print(unique_states)
# 添加哑变量列
price_date_data['is_CA'] = (price_date_data['state_id'] == 'CA').astype(int)
price_date_data['is_TX'] = (price_date_data['state_id'] == 'TX').astype(int)

struc_data=price_date_data.copy()

# 可安全填0的列
zero_fill_cols = [
    'event_count', 'cultural_count', 'religious_count', 
    'national_count', 'sporting_count',
    'snap_CA_total', 'snap_TX_total', 'snap_WI_total'
]
struc_data[zero_fill_cols] = struc_data[zero_fill_cols].fillna(0)

# 需谨慎处理的列
struc_data['is_nov_dec'] = struc_data['is_nov_dec'].fillna(0)  # 需业务确认
struc_data['sell_price'] = struc_data.groupby('id')['sell_price'].transform(
    lambda x: x.fillna(x.mean())  # 按商品ID分组填充均价
)
#最后十周作为预测和优化数据

null_counts = struc_data.isnull().sum()
print("各列空值统计:")
print(null_counts[null_counts > 0])  # 只显示有空值的列
print(struc_data.shape)


struc_data.to_csv('struc_data_without_weather.csv')

